In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install pydub

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Your token has been saved in your conf

In [ ]:
!git config --global credential.helper store

In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install torchaudio

In [ ]:
!pip install librosa

In [ ]:
!pip install jiwer

In [ ]:
from datasets import load_dataset

In [ ]:


train_dataset = load_dataset('mozilla-foundation/common_voice_13_0', "sw", split='train' )
validation_dataset = load_dataset('mozilla-foundation/common_voice_13_0', "sw", split='validation')
test_dataset = load_dataset('mozilla-foundation/common_voice_13_0', "sw", split='test')


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
train_dataset

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 34980
})

In [ ]:
## Function to show random elements from the train, validate, and test dataset

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(train_dataset)

,client_id,path,audio,sentence,up_votes,down_votes,age,gender,accent,locale,segment,variant
0,0e19d024efd1bd00fdcd20209935f3472caeec3985e168c691179a4fe8768492a0cc3de66732c2ffbfaca7fc9ea345e3870c0c072bc0ad5ffbcd8602a168894f,/root/.cache/huggingface/datasets/downloads/extracted/4a443506d39b8b48fcc983dd5b38f1616acba3909b5a917d6302bbbe14aec72e/sw_train_0/common_voice_sw_29936537.mp3,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/4a443506d39b8b48fcc983dd5b38f1616acba3909b5a917d6302bbbe14aec72e/sw_train_0/common_voice_sw_29936537.mp3', 'array': [0.0, -5.968558980384842e-13, -1.0231815394945443e-12, -7.673861546209082e-13, -3.410605131648481e-13, -2.5579538487363607e-13, -9.947598300641403e-14, -3.836930773104541e-13, -4.689582056016661e-13, -4.547473508864641e-13, -8.526512829121202e-14, 4.263256414560601e-13, 8.526512829121202e-13, 9.094947017729282e-13, 6.252776074688882e-13, -4.973799150320701e-13, -1.6200374375330284e-12, -2.2453150450019166e-12, -2.5721647034515627e-12, -3.154809746774845e-12, -4.092726157978177e-12, -4.447997525858227e-12, -4.206412995699793e-12, -2.8279600883251987e-12, -1.2789769243681803e-13, 3.083755473198835e-12, 5.6132876125047915e-12, 6.750155989720952e-12, 6.423306331271306e-12, 5.9969806898152456e-12, 6.45172804070171e-12, 6.94910795573378e-12, 5.4285465012071654e-12, 1.7905676941154525e-12, -2.7569058147491887e-12, -6.536993168992922e-12, -7.389644451905042e-12, -3.751665644813329e-12, 1.5347723092418164e-12, 5.201172825763933e-12, 5.4001247917767614e-12, 3.353761712787673e-12, 1.4068746168049984e-12, 3.296918293926865e-12, 1.0402345651527867e-11, 1.8630430531629827e-11, 2.523847797419876e-11, 3.007016857736744e-11, 3.182165642101609e-11, 3.069544618483633e-11, 2.984279490192421e-11, 2.8535396268125623e-11, 2.3021584638627246e-11, 1.5624834759364603e-11, 1.56035184772918e-11, 2.8251179173821583e-11, 5.077538389741676e-11, 7.229061793623259e-11, 7.958078640513122e-11, 6.787104211980477e-11, 4.7350567911053076e-11, 3.794298208958935e-11, 5.444888984129648e-11, 9.015366231324151e-11, 1.1480949524411699e-10, 9.612222129362635e-11, 2.560796019679401e-11, -7.199218998721335e-11, -1.4559020655724453e-10, -1.4870238373987377e-10, -7.861444828449748e-11, 1.361399881716352e-11, 5.680078629666241e-11, 9.78772618509538e-12, -1.0643930181686301e-10, -2.163460521842353e-10, -2.381170816079248e-10, -1.4159695638227277e-10, 3.041122909053229e-11, 1.8991386241395958e-10, 2.616502570162993e-10, 2.3203483578981832e-10, 1.5412382481372333e-10, 1.035864727327862e-10, 1.2497736179284402e-10, 2.028741619142238e-10, 2.771685103652999e-10, 2.9383784294623183e-10, 2.452509306749562e-10, 1.7024603948812e-10, 1.197690835397225e-10, 1.2249756764504127e-10, 1.6854073692229576e-10, 2.2606627680943348e-10, 2.7267788027529605e-10, 3.072386789426673e-10, 3.3514879760332406e-10, 3.5083758120890707e-10, 3.319655661471188e-10, 2.531663767513237e-10, ...], 'sampling_rate': 48000}",Abeid Amani Karume akiwa na umri wa miaka minne,2,0,twenties,male,,sw,,
1,729c66e7f38e458f1704244f7ab4b0f81a2af1eedaa9e7f746a43ba1536666854acde46b7cfa744e77ae56660cb6a05f07bff4d2cce5b89c42e797dc5d3b1eb2,/root/.cache/huggingface/datasets/downloads/extracted/4a443506d39b8b48fcc983dd5b38f1616acba3909b5a917d6302bbbe14aec72e/sw_train_0/common_voice_sw_30631040.mp3,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/4a443506d39b8b48fcc983dd5b38f1616acba3909b5a917d6302bbbe14aec72e/sw_train_0/common_voice_sw_30631040.mp3', 'array': [7.105427357601002e-15, -3.552713678800501e-14, 5.684341886080802e-14, 1.7053025658242404e-13, 3.552713678800501e-13, 4.831690603168681e-13, 4.405364961712621e-13, 3.552713678800501e-13, 2.1671553440683056e-13, -1.1368683772161603e-13, -5.080380560684716e-13, -7.958078640513122e-13, -9.876544027065393e-13, -1.0977885267493548e-12, -1.1652900866465643e-12, -1.3713474800169934e-12, -1.623590151211829e-12, -1.5418777365994174e-12, -1.1155520951433573e-12, -7.673861546209082e-13, -6.679101716144942e-13, -4.54747350

In [ ]:
columns_to_remove = ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"]
train_dataset = train_dataset.remove_columns(columns_to_remove)
validation_dataset = validation_dataset.remove_columns(columns_to_remove)
test_dataset = test_dataset.remove_columns(columns_to_remove)


In [ ]:
##Regular expression to remove symbols since it does not map to any phoneme in the Swahili language

In [ ]:
import re

# Modify the chars_to_remove_regex pattern to include the additional symbols
chars_to_remove_regex = r'[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\(\)\*\=\_`\[\]\/\*°ː’•…]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

cv_swahili_train = train_dataset.map(remove_special_characters)
cv_swahili_validate = validation_dataset.map(remove_special_characters)
cv_swahili_test = test_dataset.map(remove_special_characters)


In [ ]:
def replace_extra_characters(batch, column_name="sentence"):
    character_replacements = {
        'µ': 'u',
        'á': 'a',
        'â': 'a',
        'ã': 'a',
        'å': 'a',
        'é': 'e',
        'è': 'e',
        'ë': 'e',
        'í': 'i',
        'ï': 'i',
        'ñ': 'n',
        'ó': 'o',
        'ö': 'o',
        'ø': 'o',
        'ú': 'u',
        'š': 's',
        'ū': 'u',
        'μ': 'u',
        'ụ': 'u'
    }

    for original, replacement in character_replacements.items():
        batch[column_name] = re.sub(re.escape(original), replacement, batch[column_name])

    # Remove multiple dots and tabs
    batch[column_name] = re.sub(r'\.\.\.+', '', batch[column_name])
    batch[column_name] = re.sub(r'\t', '', batch[column_name])

    return batch

cv_swahili_train = cv_swahili_train.map(replace_extra_characters)
cv_swahili_validate = cv_swahili_validate.map(replace_extra_characters)
cv_swahili_test = cv_swahili_test.map(replace_extra_characters)


In [ ]:
## Extracting the Swahili phoneme from the dataset transcript


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}



vocab_train = cv_swahili_train.map(extract_all_chars, batched=True, batch_size=-1, remove_columns=cv_swahili_train.column_names)
vocab_validate = cv_swahili_validate.map(extract_all_chars, batched=True, batch_size=-1, remove_columns=cv_swahili_validate.column_names)
vocab_test = cv_swahili_test.map(extract_all_chars, batched=True, batch_size=-1, remove_columns=cv_swahili_test.column_names)


In [ ]:
# Convert "vocab" column from each dataset to sets and union them
vocab_set_train = set(vocab_train["vocab"][0])
vocab_set_validate = set(vocab_validate["vocab"][0])
vocab_set_test = set(vocab_test["vocab"][0])

# Merge vocabularies
vocab_set = vocab_set_train | vocab_set_validate | vocab_set_test

# Convert the result back to a list
vocab_list = list(vocab_set)


In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict.pop('י', None)
vocab_dict

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
print(vocab_dict)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '|': 0}


In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

29

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from huggingface_hub import create_repo
create_repo("finetuning-wav2vec-large-swahili-asr-model_v10")

In [ ]:
repo_name = "finetuning-wav2vec-large-swahili-asr-model_v10"
tokenizer.push_to_hub(repo_name)

In [ ]:
## Create Wav2VecFeatureExtractor


In [ ]:
Speech is a continuous signal, and, to be treated by computers, it first has to be discretized, which is usually called sampling.
The sampling rate hereby plays an important role since it defines how many data points of the speech signal are measured per second.
Therefore, sampling with a higher sampling rate results in a better approximation of the real speech signal but also necessitates more values per second.

A pretrained checkpoint expects its input data to have been sampled more or less from the same distribution as the data it was trained on.
The same speech signals sampled at two different rates have a very different distribution. For example, doubling the sampling rate results in data points
 being twice as long. Thus, before fine-tuning a pretrained checkpoint of an ASR model,
 it is crucial to verify that the sampling rate of the data that was used to pretrain the model matches the sampling rate of the dataset
 used to fine-tune the model.

XLS-R was pretrained on audio data of Babel, Multilingual LibriSpeech (MLS), Common Voice, VoxPopuli, and VoxLingua107 at a sampling rate of 16kHz
 Common Voice, in its original form, has a sampling rate of 48kHz, thus we will have to downsample the fine-tuning data to 16kHz in the following.

XLS-R was pretrained on audio data of Babel, Multilingual LibriSpeech (MLS), Common Voice, VoxPopuli, and VoxLingua107 at a sampling rate of 16kHz.
Common Voice, in its original form, has a sampling rate of 48kHz, thus we will have to downsample the fine-tuning data to 16kHz in the following.

A Wav2Vec2FeatureExtractor object requires the following parameters to be instantiated:

1. Feature_size: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies,
 the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal

2. Sampling_rate: The sampling rate at which the model is trained on.
padding_value: For batched inference, shorter inputs need to be padded with a specific value

3. Do_normalize: Whether the input should be zero-mean-unit-variance normalized or not. Usually, speech models perform better when normalizing the input
return_attention_mask: Whether the model should make use of an attention_mask for batched inference.
In general, XLS-R models checkpoints should always use the attention_mask.

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)


In [ ]:
##preprocess data

In [ ]:
cv_swahili_train[0]["audio"]

In [ ]:
from datasets import load_metric, Audio
cv_swahili_train = cv_swahili_train.cast_column("audio", Audio(sampling_rate=16_000))
cv_swahili_validate = cv_swahili_validate.cast_column("audio", Audio(sampling_rate=16_000))
cv_swahili_test = cv_swahili_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
cv_swahili_train[0]["audio"]

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(cv_swahili_train)-1)

print(cv_swahili_train[rand_int]["sentence"])
ipd.Audio(data=cv_swahili_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

In [ ]:
rand_int = random.randint(0, len(cv_swahili_train)-1)

print("Target text:", cv_swahili_train[rand_int]["sentence"])
print("Input array shape:", cv_swahili_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", cv_swahili_train[rand_int]["audio"]["sampling_rate"])

In [ ]:
## Preprocess the dataset to the format expected by Wave2Vec2ForCTC leveraging Wav2Vec2Processor

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
cv_swahili_train = cv_swahili_train.map(prepare_dataset, remove_columns=cv_swahili_train.column_names)
cv_swahili_validate = cv_swahili_validate.map(prepare_dataset, remove_columns=cv_swahili_validate.column_names)
cv_Swahili_test = cv_swahili_test.map(prepare_dataset, remove_columns=cv_swahili_test.column_names)

In [ ]:
from datasets import concatenate_datasets

# Concatenate cv_swahili_train and cv_swahili_validate
combined_train_validate = concatenate_datasets([cv_swahili_train, cv_swahili_validate])

In [ ]:
##training

In [ ]:
The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's Trainer
 for which we essentially need to do the following:

Define a data collator. In contrast to most NLP models, XLS-R has a much larger input length than output length.
E.g., a sample of input length 50000 has an output length of no more than 100. Given the large input sizes,
it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the
longest sample in their batch and not the overall longest sample. Therefore, fine-tuning XLS-R requires a special padding data collator, which we will define below

Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a compute_metrics function accordingly

Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

Define the training configuration.

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
rom transformers import Wav2Vec2ForCTC, AutoModelForCTC

# model = Wav2Vec2ForCTC.from_pretrained(
#     "facebook/wav2vec2-xls-r-300m",
#     ctc_loss_reduction="mean",
#     pad_token_id=processor.tokenizer.pad_token_id,
#     vocab_size=len(processor.tokenizer),
# )
#model = AutoModelForCTC.from_pretrained("AntonyG/fine-tune-wav2vec2-large-xls-r-1b-sw")
model = AutoModelForCTC.from_pretrained("Joshua-Abok/finetuned_wav2vec_asr")
# model.enable_input_require_grads()

In [ ]:
model.to("cuda")
model.freeze_feature_extractor()

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"


In [ ]:
# from peft import LoraConfig, get_peft_model, TaskType

# lora_config = LoraConfig(
#     r=16, # Rank
#     lora_alpha=32,
#     target_modules=["q_proj", "v_proj"],
#     lora_dropout=0.05,
#     bias="none",
# )
# model = get_peft_model(model, lora_config)
print(print_number_of_trainable_model_parameters(model))

In [ ]:
from transformers import TrainingArguments, get_linear_schedule_with_warmup

# Check if you are running on a CUDA-enabled device before enabling FP16
import torch
if torch.cuda.is_available():
    fp16_enabled = True
else:
    fp16_enabled = False
    print("CUDA device not available. Disabling FP16.")

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=15,
  gradient_checkpointing=True,
  fp16=fp16_enabled,  # Enable FP16 only if CUDA is available
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
  remove_unused_columns=False
)

# Define the scheduler parameters
num_warmup_steps = int(training_args.max_steps * 0.1)

In [ ]:
## Pass all instances to Trainer, ready for training

In [ ]:
from transformers import Trainer
# from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import AdamW

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=combined_train_validate,
    eval_dataset=cv_Swahili_test,
    tokenizer=processor.feature_extractor,
)

# Create the learning rate scheduler
optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)
# scheduler = OneCycleLR(
#     optimizer,
#     total_steps=1000,
#     epochs=training_args.num_train_epochs,
#     steps_per_epoch=len(cv_swahili_train),
#     pct_start=0.3,  # Percentage of the cycle that's a rising learning rate
#     max_lr=4*training_args.learning_rate,    # peak learning rate
#     final_div_factor=25.0,  # in the end LR will decay to initial_LR/final_div_factor
# )
total_steps = 1000
scheduler = CosineAnnealingLR(
    optimizer,
    T_max=total_steps,
    eta_min=0,  # Minimum learning rate
)

# Define a function to update the learning rate during training
def update_lr():
  scheduler.step()

In [ ]:
# Check if this is the correct implementation of the CosineAnnealingLR. It looks like a better learning rate schedulers.
# from torch.optim import AdamW
# from torch.optim.lr_scheduler import CosineAnnealingLR

# optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)
# total_steps = 1000
# epochs = training_args.num_train_epochs
# steps_per_epoch = len(cv_swahili_train)

# scheduler = CosineAnnealingLR(
#     optimizer,
#     T_max=total_steps,
#     eta_min=0,  # Minimum learning rate
# )

In [ ]:
To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped
 as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model,
 which is why the group_tokens=False parameter has to be passed. If we wouldn't pass this parameter a word like "hello" would incorrectly be encoded,
 and decoded as "helo". 2 2 The blank token allows the model to predict a word, such as "hello" by
 forcing it to insert the blank token between the two l's. A CTC-conform prediction of "hello" of our
  model would be [PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD].



In [ ]:
##Training

In [ ]:
# Training loop
for epoch in range(training_args.num_train_epochs):
    trainer.train()
    update_lr()
    trainer.evaluate()

In [ ]:
!nvidia-smi

In [ ]:
trainer.push_to_hub()    #had quotes -> ("")

In [ ]:
os.chdir("../")

In [ ]:
!pwd

In [ ]:
# !git clone "https://huggingface.co/Joshua-Abok/finetuning-wav2vec-large-swahili-asr-model_v9"

In [ ]:
# !pwd

In [ ]:
# os.chdir("/content/finetuning-wav2vec-large-swahili-asr-model_v9")

In [ ]:
# !pwd

In [ ]:
!ls

In [ ]:
%cd "/content/finetuning-wav2vec-large-swahili-asr-model_v9/"

In [ ]:
!git status

In [ ]:
import json

with open("/var/log/colab-jupyter.log", "r") as fo:
  for line in fo:
    print(json.loads(line)['msg'])

In [ ]:
#from pydub import AudioSegment
#from datasets import load_dataset

#def convert_mp3_to_wav(input_dataset, output_dataset):

    #for i in range(len(input_dataset)):
        # Load MP3 audio file from input dataset
        #mp3_audio_path = input_dataset[i]['path']
        #audio = AudioSegment.from_mp3(mp3_audio_path)

        # Export audio to WAV format and update the output dataset with the WAV file path
        #wav_audio_path = mp3_audio_path.replace(".mp3", ".wav")
        #audio.export(wav_audio_path, format="wav")

        # Update the output dataset with the WAV file path
        #output_dataset[i]['path'] = wav_audio_path

# Example usage
# Load input datasets
#train_set = train_dataset
#validation_set = validation_dataset
#test_set = test_dataset

# Create output datasets with the same structure as the input datasets
#converted_train_dataset = train_dataset.copy()
#converted_validation_dataset = validation_dataset.copy()
#converted_test_dataset = test_dataset.copy()

# Convert MP3 to WAV and update the output datasets
#convert_mp3_to_wav(train_set, converted_train_dataset)
#convert_mp3_to_wav(validation_set, converted_validation_dataset)
#convert_mp3_to_wav(test_set, converted_test_dataset)

In [ ]:
#import json
#from tqdm import tqdm

#validation_manifest = validation_dataset.json
#test_manifest = test_dataset.json
#train_manifest =train_dataset.json
#def compute_char_counts(manifest):
  #char_counts = {}
  #with open(manifest, 'r') as fn_in:
      #for line in tqdm(fn_in, desc="Compute counts.."):
          #line = line.replace("\n", "")
          #data = json.loads(line)
          #text = data["text"]
          #for word in text.split():
              #for char in word:
                  #if char not in char_counts:
                      #char_counts[char] = 1
                  #else:
                      #char_counts[char] += 1
  #return char_counts

#char_counts = compute_char_counts(train_manifest)

#threshold = 10
#trash_char_list = []

#for char in char_counts:
  #if char_counts[char] <= threshold:
      #trash_char_list.append(char)
#Let's check:

3print(trash_char_list)